## 控制迷宫寻宝机器人

在这个项目中，你将使用刚刚学到的知识，尝试根据要求，编写代码，来控制一个机器人，在模拟环境中行走，并找到目标宝藏。

机器人所在的模拟环境中，会包含这样几个因素：机器人的起点、障碍物、宝藏箱。你的任务包括：

1. 分析模拟环境的数据
2. 控制机器人随机行动
3. （可选）控制机器人走到终点


* 一个良好的含有注释的代码，可以让你的程序可读性更高，尝试为你自己的代码添加相应的注释。

---

---

## 第一节 分析模拟环境的数据

首先，只有足够了解机器人所在的环境，我们的机器人才能成功找到目标宝藏，因此首先我们来对机器人所在环境的数据进行分析。在这个部分，会考察你对数据结构、控制流的了解。


### 1.1 理解模拟环境数据的储存格式

首先我们思考这样的问题：如何存储模拟环境的数据呢？

我们将我们的模拟环境抽象成一个格子世界，每个格子按照坐标编号进行标记；每个格子中会有四个情况，分别为普通格子（可通行）、机器人的起点（可通行）、障碍物（不可通行）、宝藏箱（目标点）。例如，一个模拟环境就可以抽象成3行4列的格子世界，并按这按这样的方法进行存储：
```
environment = [[0,0,0,2], 
               [1,2,0,0],
               [0,2,3,2]]
```
我们用了一个列表来保存虚拟世界的数据。外层列表中的每一个元素依然是一个列表，它代表模拟环境中每行的数据。而对于这个列表中的每个元素都是一个数，它们的含义是：
- 0: 普通格子（可通行）
- 1: 机器人的起点（可通行）
- 2: 障碍物（不可通行）
- 3: 宝藏箱（目标点）

那么，根据上述的数据，这个迷宫的第二行第一列，是我们机器人的起点。

__注：我们描述的迷宫的坐标位置（第一行第一列），和迷宫下标索引的值（如 `(0,0)`）是不一样的，请注意下标的问题。__


如下的代码，使用了辅助函数，读取了模拟环境的数据，并保存在了 `env_data` 变量中。


In [1]:
import helper

env_data = helper.fetch_maze()

maze-id 1-1532800243
[[0, 0, 0, 0, 0, 2, 3],
 [1, 2, 2, 2, 0, 2, 0],
 [2, 0, 0, 0, 0, 2, 0],
 [2, 2, 2, 2, 0, 0, 0]]


---


**任务1：**在如下代码中，请写代码获得这些值：

1. 模拟环境的长和宽
2. 模拟环境中第3行第6列元素

In [2]:
#TODO 1模拟环境的行数
rows = len(env_data)

#TODO 2模拟环境的列数
columns = len(env_data[0])

#TODO 3取出模拟环境第三行第六列的元素
row_3_col_6 = env_data[2][5]

print("迷宫共有", rows, "行", columns, "列，第三行第六列的元素是", row_3_col_6)

迷宫共有 4 行 7 列，第三行第六列的元素是 2


---

## 1.2 分析模拟环境数据

接着我们需要对模拟环境的中的数据进行分析。请根据如下的指示，计算相应的值。

---

**任务2：**在如下代码中，请计算模拟环境中，第一行和第三列的障碍物个数。

提示：*可以用循环完成。*

In [3]:
#TODO 4计算模拟环境中，第一行的的障碍物个数。
number_of_barriers_row1 = 0

for e in env_data[0]:
    if e == 2:
        number_of_barriers_row1 += 1

#TODO 5计算模拟环境中，第三列的的障碍物个数。
number_of_barriers_col3 = 0
for e in env_data:
    if e[2] == 2:
        number_of_barriers_col3 += 1
print("迷宫中，第一行共有", number_of_barriers_row1, "个障碍物，第三列共有", number_of_barriers_col3, "个障碍物。")

迷宫中，第一行共有 1 个障碍物，第三列共有 2 个障碍物。


---

**任务3：**在如下代码中：

1. 创建一个名为 `loc_map` 的字典，它有两个键值，分别为 `start` 和 `destination`，对应的值分别为起点和目标点的坐标，它们以如 `(1,1)` 的形式保存为元组。
2. 从字典中取出 `start` 对应的值，保存在 `robot_current_loc` 对应的变量中，这个变量表示小车现在的位置。

In [4]:
loc_map = dict() #TODO 6按照上述要求创建字典
#初始化变量，用来保存位置
row = 0
col = 0
#遍历集合，找到目标位置
for e1 in env_data:
    col = 0
    for e2 in e1:
        if e2 == 1:
            loc_map["start"] = (row, col)
        if e2 == 3:
            loc_map["destination"] = (row, col)
        col += 1
    row += 1
        
robot_current_loc = loc_map["start"] #TODO 7保存机器人当前的位置


---

---

## 第二节 控制机器人随机漫步

在这一步中，你需发出指令，控制机器人在环境中随机行动。它会考察你对控制流、调用函数的知识。



## 2.1 控制机器人行动

我们的机器人能够执行四个动作：向上走 `u`、向下走 `d`、向左走 `l`、向右走 `r`。但是，由于有障碍，很多时候机器人的行动并不能成功。所以在这里，你需要实现一个函数，来判断机器人在某个位置，执行某个移动动作是否可行。

---

**任务4：**在下方代码中，实现名为 `is_move_valid_special` 的函数，它有两个输入，分别为机器人所在的位置坐标 `loc`，以及即将执行的动作 `act`，如 `(1,1)` 及 `u`。接着它的返回是一个布尔值，表明小车在 `loc` 位置下，是否可以执行动作 `act`。


提示1：*可以读取上方定义的 `env_data` 变量，来读取模拟环境的数据。*

提示2：*在实现函数后，请删去下方的 `pass` 代码。*

提示3：*我们需要处理边界的情况，即机器人走到了虚拟环境边界时，是不能够走出虚拟环境的。*

In [5]:
def is_move_valid_sepcial(loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 8
    if act == "u":
        #如果当前行是第一行或者上一行是2，那么不可走，否则可走
        if loc[0] == 0 or env_data[loc[0] - 1][loc[1]] == 2:
            return False
        else:
            return True
        
    elif act == "d":
        #如果当前行是最后一行或者下一行是2，那么不可走，否则可走
        if loc[0] == rows - 1 or env_data[loc[0] + 1][loc[1]] == 2:
            return False
        else:
            return True
    elif act == "l":
        #如果当前列是第一列或者前一列是2，那么不可走，否则可走
        if loc[1] == 0 or env_data[loc[0]][loc[1] - 1] == 2:
            return False
        else:
            return True
    else:
        #R
        #如果当前列是最后一列或下一列是2，那么不可走，否则可走
        if loc[1] == (columns - 1) or (env_data[loc[0]][loc[1] + 1]) == 2:
            return False
        else:
            return True  

---

**任务5：**在下方代码中，重新实现一个名为 `is_move_valid` 的函数，它有三个输入，分别为模拟环境的数据 `env_data`、机器人所在的位置坐标 `loc`、以及即将执行的动作 `act`。它的返回值与此前一样，是一个布尔值，表明小车在给定的虚拟环境中的 `loc` 位置下，是否可以执行动作 `act`。

In [6]:
def is_move_valid(env_data, loc, act):
    """
    Judge wether the robot can take action act
    at location loc.
    
    Keyword arguments:
    env -- list, the environment data
    loc -- tuple, robots current location
    act -- string, robots meant action
    """
    #TODO 9
    #获取环境的长和宽
    rows = len(env_data)
    columns = len(env_data[0])
    if act == "u":
        #如果当前行是第一行或者上一行是2，那么不可走，否则可走
        if loc[0] == 0 or env_data[loc[0] - 1][loc[1]] == 2:
            return False
        else:
            return True
        
    elif act == "d":
        #如果当前行是最后一行或者下一行是2，那么不可走，否则可走
        if loc[0] == rows - 1 or env_data[loc[0] + 1][loc[1]] == 2:
            return False
        else:
            return True
    elif act == "l":
        #如果当前列是第一列或者前一列是2，那么不可走，否则可走
        if loc[1] == 0 or env_data[loc[0]][loc[1] - 1] == 2:
            return False
        else:
            return True
    else:
        #R
        #如果当前列是最后一列或下一列是2，那么不可走，否则可走
        if loc[1] == (columns - 1) or (env_data[loc[0]][loc[1] + 1]) == 2:
            return False
        else:
            return True

---

**任务6：**请回答：**在任务4及任务5中的实现的两个函数中，`env_data` 这个变量有什么不同？**

提示：_可以尝试从变量作用域的角度回答该问题。_

**回答：** （请在这里填写你的回答）

在任务4中，使用的env_data是全局的，在该类中的任何一个地方都可以访问
而在任务5中，使用的env_data是局部的，只有在方法内部才可以使用，外部不可以访问，当方法执行完之后，对象的引用已经不存在了。

---

## 2.2 机器人可行动作

---

**任务7：**编写一个名为 `valid_actions` 的函数。它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`，输出是一个列表，表明机器人在这个位置所有的可行动作。

提示：*可以尝试调用上方定义的`is_move_valid`函数。*


In [7]:
## TODO 10 从头定义、实现你的函数
def valid_actions(env_data, loc):
    #定义动作列表
    all_action_list = ["u", "l", "r", "d"]
    #定义一个列表,用于返回当前位置可热的动作列表
    action_list = list() 
    #遍历动作全列表，分别传入左上右下四个方向的Action,得到是否可移动，最终把结果添加进新的列表
    for act in all_action_list:
        if is_move_valid(env_data, loc, act):
            action_list.append(act)
        
    return action_list


    

---

## 2.3 移动机器人

当机器人收到一个动作的时候，你机器人的位置应发生相应的变化。

**任务8：**编写一个名为 `move_robot` 的函数，它有两个输入，分别为机器人当前所在的位置 `loc` 和即将执行的动作 `act`。接着会返回机器人执行动作之后的新位置 `new_loc`。

In [8]:
##TODO 11 从头定义、实现你的函数
def move_robot(loc, act):
    #定义一个空的元姐
    new_loc = tuple()
    #根据当前的Action给元组赋值
    if act == "u":
        new_loc = (loc[0] - 1, loc[1])
    elif act == "d":
        new_loc = (loc[0] + 1, loc[1])
    elif act == "l":
        new_loc = (loc[0], loc[1] - 1)
    else:
        new_loc = (loc[0], loc[1] + 1)
    return new_loc

---

## 2.4 随机移动机器人

接着，我们尝试在虚拟环境中随机移动机器人，看看会有什么效果。

**任务9：**编写一个名为 `random_choose_actions` 的函数，它有两个输入，分别为虚拟环境的数据 `env_data`，以及机器人所在的位置 `loc`。机器人会执行一个300次的循环，每次循环，他会执行以下任务：

1. 利用上方定义的 `valid_actions` 函数，找出当前位置下，机器人可行的动作；
2. 利用 `random` 库中的 `choice` 函数，从机器人可行的动作中，随机挑选出一个动作；
3. 接着根据这个动作，利用上方定义的 `move_robot` 函数，来移动机器人，并更新机器人的位置；
4. 当机器人走到终点时，输出“在第n个回合找到宝藏！”。

提示：如果机器人无法在300个回合内找到宝藏的话，试试看增大这个数字，也许会有不错的效果 :P

In [9]:
import random 
##TODO 12 从头实现你的函数
def random_choose_actions(env_data, loc):
    #定义Action列表。
    actions = ["u", "l", "r", "d"]
    #定义一个新的变量，存当前位置。
    cur_loc = loc
    #定义一个300次的循环
    for i in range(0, 300):
        #找到当前可以执行的动作
        usefull_act = valid_actions(env_data, cur_loc)
        #随便选择一个可执行的动作
        random_act = random.choice(usefull_act)
        #更新当前的位置
        cur_loc = move_robot(cur_loc, random_act)
        if (env_data[cur_loc[0]][cur_loc[1]]) == 3:
            print("在第{}个回合找到宝藏！".format(i))
            break

In [10]:
# 运行
random_choose_actions(env_data, robot_current_loc)

在第118个回合找到宝藏！



---

---

## （可选）第三节 控制机器人走到终点

## 3.1 控制机器人走到终点

在这里，你将综合上述的知识，编码控制机器人走到终点。这个任务对刚刚入门的你来说可能有些挑战，所以它是一个选做题。

**任务10**：尝试实现一个算法，能够对给定的模拟环境，输出机器人的行动策略，使之能够走到终点。

提示：_你可以尝试参考 [A星算法](https://zh.wikipedia.org/wiki/A*%E6%90%9C%E5%AF%BB%E7%AE%97%E6%B3%95) 。_
以及以下参考资料：
* https://baike.baidu.com/item/A%2A算法

* https://blog.csdn.net/hitwhylz/article/details/23089415

In [14]:
##TODO 13 实现你的算法
##TODO 13 实现你的算法
import random
def has_obstacle(env_data, cur_loc, des_loc):
    """
    前方是否有障碍物
    参数：
    env_data:环境
    cur_loc:当前位置，格式(0, 1)
    des_loc:目标位置，格式(0, 1)
    返回：
    Ture或False
    """
    #目标在左边时
    if des_loc[1] < cur_loc[1]:
        return has_obstacle(env_data, des_loc[1] + 1, cur_loc[1] - 1)
    elif des_loc[1] > cur_loc[1]:
         return has_obstacle(env_data, cur_loc[1] + 1, des_loc[1] - 1)
    else:
        return False


def has_obstacle(env_data, start, end):
    """
    前方是否有障碍物
    参数：
    env_data:环境
    start:开始列
    end:结束列（包括)
    返回：
    Ture或False
    """
    #区间是否有障碍物
    for i in range(len(env_data)):
        for j in range(start, end + 1):
            if env_data[i][j] == 3:
                return True
    return False

def get_cur_loc(env_data):
    """
    获取当前的位置
    参数：
    env_data:环境
    返回：
    当前位置，格式(1, 0)
    """
    for i in range(len(env_data)):
        for j in range(len(env_data[0])):
            if env_data[i][j] == 1:
                return (i, j)


def get_des_loc(env_data):
    """
    获取宝藏的位置
    参数：
    env_data:环境
    返回：
    当前位置，格式(1, 0)
    """
    for i in range(len(env_data)):
        for j in range(len(env_data[0])):
            if env_data[i][j] == 3:
                return (i, j)

def contains_loc(paths, loc):
    """
    当前的路径是否走过，走过返回True, 否则False
    """
    for k, v in paths.items():
        for e in v:
            if e == loc:
                return True
    return False

def move_robot(loc, act):
    #定义一个空的元姐
    new_loc = tuple()
    #根据当前的Action给元组赋值
    if act == "u":
        new_loc = (loc[0] - 1, loc[1])
    elif act == "d":
        new_loc = (loc[0] + 1, loc[1])
    elif act == "l":
        new_loc = (loc[0], loc[1] - 1)
    else:
        new_loc = (loc[0], loc[1] + 1)
    return new_loc


def can_move(paths, env_data, loc, act):
    print("can_move, loc: {}".format(loc))
    """
     是否可以走。
    """
    #如果已经走过的，返回False.
    if contains_loc(paths, move_robot(loc, act)):
        print("return 1")
        return False

    if loc[0] < 0 or loc[0] > len(env_data) - 1 or loc[1] < 0 or loc[1] > len(env_data[0]) - 1:
        print("return 2")
        return False

    #获取环境的长和宽
    rows = len(env_data)
    columns = len(env_data[0])

    if act == "u":
        #如果当前行是第一行或者上一行是2，那么不可走，否则可走
        if loc[0] <= 0 or env_data[loc[0] - 1][loc[1]] == 2:
            return False
        else:
            return True

    elif act == "d":
        #如果当前行是最后一行或者下一行是2，那么不可走，否则可走
        if loc[0] >= rows - 1 or env_data[loc[0] + 1][loc[1]] == 2:
            return False
        else:
            return True
    elif act == "l":
        #如果当前列是第一列或者前一列是2，那么不可走，否则可走
        if loc[1] <= 0 or env_data[loc[0]][loc[1] - 1] == 2:
            return False
        else:
            return True
    else:
        #R
        #如果当前列是最后一列或下一列是2，那么不可走，否则可走
        if loc[1] >= (columns - 1) or (env_data[loc[0]][loc[1] + 1]) == 2:
            return False
        else:
            return True

#开始分支
def start_branch(paths, cur_loc, des_loc, branch_index, act1, act2) :
    #原始分支
    print("分支{}".format(branch_index))
    ori_loc = move_robot(cur_loc, act1)
    paths[str(branch_index)].append((ori_loc[0], ori_loc[1]))
    move(paths, branch_index, ori_loc, des_loc)

    #新分支
    value = paths[str(branch_index)]
    branch_index = len(paths)
    new_loc = move_robot(cur_loc, act2)
    paths[str(branch_index)] = list()

    #把上一个走过的路径添加进新的路径中
    for i in value:
        paths[str(branch_index)].append(i)
    paths[str(branch_index)].append(new_loc)
    move(paths, branch_index, new_loc, des_loc)

    pass

#移动
def move_diy(paths, branch_index, cur_loc, des_loc, act_list):
    best_loc = (cur_loc[0], cur_loc[1])
    move_list = list()
    if can_move(paths, env_data, cur_loc, act_list[1]) and can_move(paths, env_data, cur_loc, act_list[2]):
        start_branch(paths, cur_loc, des_loc, branch_index, act_list[1], act_list[2])
    else:
        if can_move_ignore_history(env_data, cur_loc, act_list[0]):
            move_list.append(move_robot(cur_loc, act_list[0]))
        if can_move_ignore_history(env_data, cur_loc, act_list[1]):
            move_list.append(move_robot(cur_loc, act_list[1]))
        if can_move_ignore_history(env_data, cur_loc,act_list[2]):
            move_list.append(move_robot(cur_loc, act_list[2]))
        if can_move_ignore_history(env_data, cur_loc, act_list[3]):
            move_list.append(move_robot(cur_loc, act_list[3]))
    s = 0;
    for temp_loc in move_list:
        cur_s = abs(temp_loc[1] - des_loc[1])
        if cur_s < s:
            best_loc = (temp_loc[0], temp_loc[1])
        s = cur_s
    return best_loc

#移动
def move(paths, branch_index, cur_loc, des_loc):
    print("当前分支：{}".format(branch_index))
    #循环走。
    while cur_loc != des_loc:
        act_list = tuple()
        if cur_loc[1] > des_loc[1]:
            #目标在左上
            if cur_loc[0] > des_loc[0]:
                act_list = ("l", "u", "d", "r")
            else:
                #目标在左下或同一行
                act_list = ("l", "d", "u", "r")
        else:
            #目标在右边或同一列,且在上边
            if cur_loc[0] > des_loc[0]:
                act_list = ("r", "u", "d", "l")
            else:
                #目标在下或同一行
                act_list = ("r", "d", "u", "l")
        cur_loc = move_diy(paths, branch_index, cur_loc, des_loc, act_list)
        #添加进字典
        paths[str(branch_index)].append((cur_loc[0], cur_loc[1]))

#寻找路径。
def find_path_b_star(env_data):
    """
    寻找路径,模拟B星算法，有很多地方要完善的
    """
    #获取当前的位置和目标位置
    cur_loc = get_cur_loc(env_data)
    des_loc = get_des_loc(env_data)
    #定义一个字典，用来存放路径，key是分支序号，value是这条路的坐标列表。
    paths = dict()
    #分支序号
    branch_index = 0
    print("执行B星算法，开始找路")
    paths[str(branch_index)] = [cur_loc]
    move(paths, branch_index, cur_loc, des_loc)
    #取出最佳的路径
    best_path = paths["0"]
    for k, v in paths.items():
        if len(v) < len(best_path):
            best_path = v

    print("找到{}条路".format(len(paths)))
    print("一共走了{}步".format(len(best_path) - 1))
    print("把行走的路径替换为.号(起点和终点除外),打印路径")
    for i in range(len(env_data)):
        for j in range(len(env_data[0])):
            loc = (i, j)
            if loc == cur_loc or loc == des_loc:
                print(" " + str(env_data[i][j]) + " ", end='')
                continue
            if loc in best_path:
                print(" . ", end='')
            else:
                print(" " + str(env_data[i][j]) + " ", end='')
        print(" ")
    print(best_path[0:50])
    print("\n" * 5)
    print("================================BBBBBBBBBBBBBBBBBBBBBBBBB===================================")
    print("\n" * 5)
    pass

def can_move_ignore_history(env_data, loc, act):
    """
     是否可以走。
    """
    #获取环境的长和宽
    rows = len(env_data)
    columns = len(env_data[0])

    if loc[0] < 0 or loc[0] > len(env_data) - 1 or loc[1] < 0 or loc[1] > len(env_data[0]) - 1:
        return False
    if act == "u":
        #如果当前行是第一行或者上一行是2，那么不可走，否则可走
        if loc[0] == 0 or env_data[loc[0] - 1][loc[1]] == 2:
            return False
        else:
            return True

    elif act == "d":
        #如果当前行是最后一行或者下一行是2，那么不可走，否则可走
        if loc[0] == rows - 1 or env_data[loc[0] + 1][loc[1]] == 2:
            return False
        else:
            return True
    elif act == "l":
        #如果当前列是第一列或者前一列是2，那么不可走，否则可走
        if loc[1] == 0 or env_data[loc[0]][loc[1] - 1] == 2:
            return False
        else:
            return True
    else:
        #R
        #如果当前列是最后一列或下一列是2，那么不可走，否则可走
        if loc[1] == (columns - 1) or (env_data[loc[0]][loc[1] + 1]) == 2:
            return False
        else:
            return True

#寻找路径。
def find_path_a_star(env_data):
    """
    寻找路径,模拟A星算法
    """
    #获取当前的位置和目标位置
    cur_loc = get_cur_loc(env_data)
    des_loc = get_des_loc(env_data)
    #定义字典openlist,格式key为坐标，value为估算距离
    #如:{(1, 3):20}
    open_list = dict()
    #定义close_list,存放的是坐标。如：[(1, 2), (2, 4)]
    close_list = list()
    #path列表,存放的是坐标。如：[(1, 2), (2, 4)]
    paths = list()
    print("执行A星算法，开始找路")
    open_list[cur_loc] = abs(cur_loc[1] - des_loc[1]) + abs(cur_loc[0] - des_loc[0])
    paths.append(cur_loc)
    #定义一个动作优先级列表
    if cur_loc[1] > des_loc[1]:
        #目标在左上
        if cur_loc[0] > des_loc[0]:
            act_list = ("l", "u", "d", "r")
        else:
            #目标在左下或同一行
            act_list = ("l", "d", "u", "r")
    else:
        #目标在右边或同一列,且在上边
        if cur_loc[0] > des_loc[0]:
            act_list = ("r", "u", "d", "l")
        else:
            #目标在下或同一行
            act_list = ("r", "d", "u", "l")
    #开始找节点.
    while cur_loc != des_loc:
        #附近的坐标集合
        closed_list = list()
        # act_list = get_act_list(cur_loc, des_loc)
        if can_move_ignore_history(env_data, cur_loc, act_list[0]):
            new_loc = move_robot(cur_loc, act_list[0])
            if new_loc not in close_list:
                 open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                 closed_list.append(new_loc)
        if can_move_ignore_history(env_data, cur_loc, act_list[1]):
            new_loc = move_robot(cur_loc, act_list[1])
            if new_loc not in close_list:
                 open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                 closed_list.append(new_loc)
        if can_move_ignore_history(env_data, cur_loc, act_list[2]):
            new_loc = move_robot(cur_loc, act_list[2])
            if new_loc not in close_list:
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)
        if can_move_ignore_history(env_data, cur_loc, act_list[3]):
            new_loc = move_robot(cur_loc, act_list[3])
            if new_loc not in close_list:
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)

        #把当前点从open_list中移除，加入close_list
        open_list.pop(cur_loc)
        close_list.append(cur_loc)

        #如果没有找到可用的点，那么不管有没有走过，都加入列表。
        if len(closed_list) == 0:
            if can_move_ignore_history(env_data, cur_loc, act_list[0]):
                new_loc = move_robot(cur_loc, act_list[0])
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)
            if can_move_ignore_history(env_data, cur_loc, act_list[1]):
                new_loc = move_robot(cur_loc, act_list[1])
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)
            if can_move_ignore_history(env_data, cur_loc, act_list[2]):
                new_loc = move_robot(cur_loc, act_list[2])
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)
            if can_move_ignore_history(env_data, cur_loc, act_list[3]):
                new_loc = move_robot(cur_loc, act_list[3])
                open_list[(new_loc[0],new_loc[1])] = abs(new_loc[1] - des_loc[1]) + abs(new_loc[0] - des_loc[0])
                closed_list.append(new_loc)

        #找到最优点，把这个点加入到paths中
        best_loc = closed_list[0]
        s = 0
        for temp_loc in closed_list:
            cur_s = open_list[temp_loc]
            if cur_s < s:
                best_loc = (temp_loc[0], temp_loc[1])
            s = cur_s

        paths.append(best_loc)
        cur_loc = (best_loc[0], best_loc[1])

    print("一共走了{}步".format(len(paths) - 1))
    print("把行走的路径替换为.号(起点和终点除外),打印路径")
    for i in range(len(env_data)):
        for j in range(len(env_data[0])):
            loc = (i, j)
            if loc == get_cur_loc(env_data) or loc == des_loc:
                print(" " + str(env_data[i][j]) + " ", end='')
                continue
            if loc in paths:
                print(" . ", end='')
            else:
                print(" " + str(env_data[i][j]) + " ", end='')
        print(" ")
    print("\n" * 2)
    print("=========================================AAAAAAAAAAAAAAAAAAAAAAAAAAA=======================================================")
    print("\n" * 2)
    pass

#测试
env_data =  [
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
]

#测试A星算法
for i in range(20):
    print("第{}次...".format(i + 1))
    cur_loc = get_cur_loc(env_data)
    des_loc = get_des_loc(env_data)
    env_data[cur_loc[0]][cur_loc[1]] = 0
    env_data[des_loc[0]][des_loc[1]] = 0
    row = random.randint(0, len(env_data) - 1)
    col = random.randint(0, (len(env_data[0]) - 1)/2 - 5)
    env_data[row][col] = 1
    row = random.randint(0, len(env_data) - 1)
    col = random.randint((len(env_data[0]) - 1)/2 + 5, len(env_data[0]) - 1)
    env_data[row][col] = 3
    find_path_a_star(env_data)

# #测试B星算法 未完成
# for i in range(1):
#     print("第{}次...".format(i + 1))
#     cur_loc = get_cur_loc(env_data)
#     des_loc = get_des_loc(env_data)
#     env_data[cur_loc[0]][cur_loc[1]] = 0
#     env_data[des_loc[0]][des_loc[1]] = 0
#     row = random.randint(0, len(env_data) - 1)
#     col = random.randint(0, (len(env_data[0]) - 1)/2 - 5)
#     env_data[row][col] = 1
#     row = random.randint(0, len(env_data) - 1)
#     col = random.randint((len(env_data[0]) - 1)/2 + 5, len(env_data[0]) - 1)
#     env_data[row][col] = 3
#     find_path_b_star(env_data)


第1次...
执行A星算法，开始找路
一共走了32步
把行走的路径替换为.号(起点和终点除外),打印路径
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  
 0  0  0  1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  .  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  2  .  2  2  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  2  0  0  0  0  0  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  2  0  0  0  0  0  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  2  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  .  .  .  .  .  .  2  2  2  2  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  .  .  .  .  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  



=========================================AAAAAAAAAAAAAAAAAAAAAAAAAAA=======================================================



第7次...
执行A星算法，开始找路
一共走了34步
把行走的路径替换为.号(起点和终点除外),

...

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  



=========================================AAAAAAAAAAAAAAAAAAAAAAAAAAA=======================================================



第15次...
执行A星算法，开始找路
一共走了40步
把行走的路径替换为.号(起点和终点除外)

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  .  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  1  .  .  .  .  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  2  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  2  2  2  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
 0  0  0  0  0  0  0  0  0  0  0  0  0  0

> 注意: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。